In [1]:
import sys
sys.path.append('C:\Program Files\Anaconda3\Lib\site-packages')

In [2]:
import socialsent
import numpy as np

In [3]:
import pandas as pd
import treetaggerwrapper
tagger = treetaggerwrapper.TreeTagger(TAGLANG='ru')
from tqdm import tqdm
import pymorphy2 
morph = pymorphy2.MorphAnalyzer()

In [2]:
#data = pd.read_csv('/home/nst/mount/data/share/dropbox/Dropbox/ML_sent/vk_corpus.csv')

In [3]:
#data = data.dropna(axis=0, how='any')

In [3]:
books_train = pd.read_csv('books_train.tsv', sep='\t', index_col=0)
films_train = pd.read_csv('films_train.tsv', sep='\t', index_col=0)

In [18]:
def lemmatize_texts(texts):
    lemmatized_texts = []
    for text in tqdm(texts):
        tagged_words = tagger.tag_text(text)
        tags2 = treetaggerwrapper.make_tags(tagged_words)
        lemmas = []
        for tag in tags2:
            try:
                lem = tag.lemma
                lemmas.append(lem)
            except AttributeError:
                lemmas.append(tag.word)
        lemmas = ' '.join(lemmas)
        lemmatized_texts.append(lemmas)
    return lemmatized_texts

def lemmatize_text_pymorphy(texts):
    lemmatized_texts = []
    for text in tqdm(texts):
        text = text.split()
        lemmas = []
        for word in text:
            lemma = morph.parse(word)[0].normal_form
            lemmas.append(lemma)
        lemmas = ' '.join(lemmas)
        lemmatized_texts.append(lemmas)
    return lemmatized_texts

In [4]:
def get_context(text, word, window_size):
#     print('get_context', word_list)
    word_list = text.split()
    contexts = []
    indices = [index for index, token in enumerate(word_list) if token == word]
    for word_index in indices:
        #word_index = word_list.index(word)
        word_list[word_index] = '<' + word_list[word_index] + '>'
        left_index = word_index-window_size
        right_index = word_index+window_size+1
        if left_index < 0:
            left_index = 0
        if right_index > len(word_list):
            right_index = len(word_list)
        context = word_list[left_index:right_index]
        #context[window_size] = '<' + context[window_size] + '>'
        context = ' '.join(context)
        contexts.append(context)
    return contexts

def get_all_contexts(texts, word, window_size):
    contexts = []
    for text in texts:
        context = get_context(text, word, window_size)
        contexts.extend(context)
    return contexts

In [5]:
books_train = books_train.dropna()
books_train = books_train.drop('lemmatized', axis=1)

In [19]:
book_lemmatized = lemmatize_text_pymorphy(books_train.X_train)

100%|██████████| 6736/6736 [02:15<00:00, 49.56it/s]


In [20]:
book_ser = pd.Series(book_lemmatized)
books_train.insert(loc=0, column='lemmatized', value=book_lemmatized)

In [21]:
books_train.to_csv('books_train.tsv', sep='\t')

## Книги. Контекст

__Книга__
(+)
* понравиться
* интересный
* лучший
* великолепный
* обожать
* приятный
* захватывать
* замечательный
* восторженный
* отличный
* любимый
* потрясающий
* супер
* обалденный
* восторг

(-)
* разочарование
* плохой
* банальность ?
* похуже (?)
* скомканному (?)
* ужасный / ужасно
* пресытиться
* худой
* нудно
* извращает
* средний (?)
* облом
* мусорный
* средненький (?)
* перековеркал

In [12]:
get_all_contexts(books_train.lemmatized, 'книга', 3)

['я прочитать этот <книга> в начало го',
 'я <книга> очень понравиться заставить',
 'человек думать этот <книга> в будущее помочь',
 'мочь передать идея <книга> и помочь разобраться',
 'начать читать четверь <книга> осилила а дальхой',
 '<книга> интересный но как',
 'лучший из весь <книга> ольги для я',
 '<книга> великолепный я бы',
 'правильно читать хороший <книга> прочитать страница почувствовать',
 'до полный прочтение <книга>',
 'и я отложить <книга> на потом',
 'для любитель оный <книга> прийтись по душа',
 'после прочтение другой <книга> владимира васильев я',
 'думать этот его <книга> один из самый',
 'обожать этот <книга> но как уже',
 'не верить этот <книга> вобще автор предупредить',
 'паланик великолепный этот <книга> так точно',
 'факт очень приятный <книга> в отличие от',
 'отпускаешь до конец <книга> более серьёзная категорически',
 'читать дважды этот <книга> запомниться на долго',
 'не читать великолепный <книга> в мой издание',
 'добавить ссылка на <книга> для скачива

__Сюжет__

(+)
* захватывать
* оригинальный
* интересный
* увлекательный
* красивый
* динамичный
* хороший
* непредсказуемый
* оригинальный
* потрясающий
* невероятный

(-)
* надоесть
* избитый
* предсказуемый
* сухой
* банальный
* плохой
* чудовищный
* разочаровать
* простоватый ??
* плоский
* безобразно
* невыразительно

In [13]:
get_all_contexts(books_train.lemmatized, 'сюжет', 3)

['не только от <сюжет> но и от',
 'просто потрясён слог <сюжет> постоянный настороженностью слежение',
 'настороженностью слежение за <сюжет> финал ошеломить такой',
 'интересный мысль но <сюжет> так себя',
 '<сюжет> конечно захватывать так',
 'терять детский непосредственность <сюжет> да он быть',
 'прийтись по душа <сюжет> оригинальный и очень',
 'персонаж который менять <сюжет> автор пускать под',
 'привлечь внимание к <сюжет> очень хорошо',
 'терроризм необыкновенно закрутить <сюжет>',
 'постоянный напряжение сам <сюжет> открываться на последний',
 'категорически не приемлть <сюжет> и герой просто',
 'книжка динамично закручивающийся <сюжет> до самый конец',
 'во первых основа <сюжет> принципиально ничто не',
 'в жизнь захватывать <сюжет> на фон живописный',
 'под конец закрутить <сюжет> я просто надоесть',
 'познавательно читаться легко <сюжет> не затянуть ждать',
 'иметь к продолжение <сюжет> от я крепкий',
 'роман с непредсказуемый <сюжет> о любовь конечно',
 'с другой орининаль

__Стиль__

(+)
* нравиться
* неповторимый
* живой
* замечательный
* отличный
* хороший


(-)
* ужасный
* графоманским
* занудно
* бесить
* возмутительно
* пренебрежительный
* громоздкий
* скучный
* телеграфный
* раздражать
* нудность

In [14]:
get_all_contexts(books_train.lemmatized, 'стиль', 3)

['все нравиться его <стиль> неповторимый отдельный момент',
 'отличный книга в <стиль> волкодав и возможно',
 'откровение после пропагандистский <стиль> 20-й такой книга',
 'книга написанна ужасно <стиль> автор весь время',
 'современный фантастика графоманским <стиль> получить четверка по',
 'слегка занудно в <стиль> грэма грин',
 'с очень живой <стиль> написание настолько живо',
 'самый современный по <стиль> роман мельниковый этакий',
 'от язык и <стиль> и для наслаждение',
 'особенный спасибо переводчик <стиль> чистота ощущение и',
 'бесить возмутительно тяжелый <стиль> громоздкий описание который',
 'пелевин в свой <стиль> яркий образ динамичный',
 'предсказуемее абсолютно пренебрежительный <стиль> главный мысль жизнь',
 'немного неуютный <стиль> по начало но',
 'очень понравиться понравиться <стиль> понравиться сюжет понравиться',
 'нежно тонко в <стиль> писатель',
 'в исключительно современный <стиль> сюжет предсказуемый и',
 'как это называться <стиль> написание что ли',
 'ност

In [22]:
films_train = films_train.dropna(axis=0, how='any')

In [23]:
films_train = films_train.drop('lemmatized', axis=1)

In [25]:
films_lemmatized = lemmatize_text_pymorphy(films_train.X_train)

100%|██████████| 10926/10926 [08:47<00:00, 20.70it/s]


In [26]:
films_ser = pd.Series(book_lemmatized)
films_train.insert(loc=0, column='lemmatized', value=films_lemmatized)

In [28]:
films_train.to_csv('films_train.tsv', sep='\t')

## Контекст. Фильмы
__Фильм__

(+)
* хороший
* классный
* светлый
* офигенный
* уникальный
* отличный
* качественный
* душевный
* замечательный
* достойный
* добрый
* веселый
* неординарный
* красивый
* гениальный
* суперский

(-)
* скучный
* скучища
* посредственный
* третьесортный
* испортить 
* омерзительный
* стрёмный
* разочаровать
* неинтересный

In [29]:
get_all_contexts(films_train.lemmatized, 'фильм', 3)

['наверное в это <фильм> о непростой жизнь',
 'и конец соответствующий <фильм> скучный бездарный саундтрек',
 'вирджиния мэдсный хороший <фильм> шаткий троечка впрочем',
 'вменяемый для летний <фильм> комедия в америка',
 'уж менять название <фильм> и дразнить публика',
 'какой образ интрига <фильм> прятать как золото',
 'не ранний середина <фильм> да и то',
 'ещё нужно дотерпеть <фильм> притворяться найти на',
 'ещё бегать весь <фильм> по улица и',
 'реальность воспроизвести сюжет <фильм> катастрофа показ катаклизм',
 'сторона герой новое <фильм> устроить для друг',
 '<фильм> не столько о',
 'соображение на ранний <фильм> тинто я узнавать',
 'такой план они <фильм> давать на слава',
 'правдивый и сильный <фильм> можно сколько угодный',
 'что весь советский <фильм> проникнутый идеология за',
 'стоять до последний <фильм> именно о такой',
 '<фильм> быть бы не',
 'персонаж в это <фильм> это герой игорь',
 'периодически декларировать некоторый <фильм> сниматься ради один',
 'туповатый а т

__Актер__

(+)
* замечательный
* хороший
* великий
* понравиться
* блестящий
* великолепно
* талантливый
* радовать
* красивый
* захватывать
* приятный
* шикарный 
* классный
* обаятельный

(-)
* плохой
* ужас
* бездарь
* некчемный
* отстойный
* смазливый
* некрасивый
* придурок
* непрофессиональный ??

In [31]:
get_all_contexts(films_train.lemmatized, 'актёр', 3)

['весь ничто и <актёр> хороший и сюжет',
 'кривой и игра <актёр> я не понравиться',
 'происходить хороший игра <актёр> щепотка ностальгия хороший',
 'дуэт великое русский <актёр> георгий юматов и',
 'тело и игра <актёр> особенно гоша куценко',
 'как серъёзный замечательный <актёр> втравить в такой',
 'блестящий игра <актёр> великолепный монолог но',
 'режиссёр часто суета <актёр> в кадр просто',
 'весь хорошо и <актёр> красивый но что',
 'отличный компания характерный <актёр> оригинальный да да',
 'никто не известный <актёр> иначе какой же',
 'технический исполнение хороший <актёр> сюжет на мой',
 'же он комедийный <актёр>',
 'звезда знаменитость кассовый <актёр> он нет смысл',
 'основное играть непрофессиональный <актёр> но как я',
 'и хороший игра <актёр> на мой взгляд',
 'и тот же <актёр> ли каншэнь который',
 'этот беда хороший <актёр> взяться за режиссура',
 'поэтому переметнуться из <актёр> в режиссёр как',
 'собрать прежний состав <актёр> сказать мотор и',
 'я понравиться игра <

__Саундтрек__

(+)
* замечательный
* хороший
* удаваться
* удачный
* отличный
* классный
* убойный ??
* оригинальность
* приятный
* крутой

(-)
* бездарный
* слабый 
* однообразный
* пошлятина

In [33]:
get_all_contexts(films_train.lemmatized, 'саундтрек', 3)

['фильм скучный бездарный <саундтрек> послушать ещё раз',
 'отсутствовать музыкальный сопровождение <саундтрек> и от это',
 'огромный бюджет замечательный <саундтрек> захватывать вид и',
 '<саундтрек> оценка отлично исполнительница',
 'конечно же запоминаться <саундтрек> конечно этот фильм',
 'динамичный сюжет замечательный <саундтрек> быстрый смена красивый',
 'динамика и хороший <саундтрек>',
 'я почти весь <саундтрек> теперь в плеер',
 'тянуть ханс циммер <саундтрек> он всегда удаваться',
 'образ лирический интеллигентка <саундтрек> к слово тоже',
 'даже и ещё <саундтрек> очень и очень',
 'жалеть особенно удачный <саундтрек>',
 'обрываться весь внезапный <саундтрек> бродить по забивать',
 'же очень порадовать <саундтрек> в фильм хотеть',
 'путаница изящно прикрыть <саундтрек> и закольцевать голос',
 'диалог нет настоящий <саундтрек> нет качественный детский',
 'и сыщик слабый <саундтрек> очень однообразный единственный',
 'отдельно хотеться отметить <саундтрек> современный ритм весь

## Embeddings

In [4]:
root_embed = 'D:/wiki.ru.vec' #embeddings, обученные в facebook с помощью fasttext

In [5]:
words = []

embeddings_index = {}
fo = open(root_embed, encoding = 'utf-8')
for line in tqdm(fo):
    values = line.split()
    if len(values) == 301:
        word = values[0]
        words.append(word)
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
fo.close()

1888424it [10:29, 2999.59it/s]


In [6]:
print(embeddings_index['придурок'])

[ -8.32260028e-02   9.23049971e-02   2.09020004e-01   1.18759997e-01
   2.69109998e-02   3.57710004e-01  -1.01060003e-01  -4.26220000e-01
   3.00880000e-02  -2.59829998e-01   4.00119990e-01   4.69550014e-01
  -6.39699996e-02  -1.49069995e-01  -2.33779997e-01   4.21250015e-02
   2.93980002e-01  -9.64220017e-02   3.32650006e-01  -1.20889999e-01
   1.15379997e-01   4.16069999e-02  -2.56299991e-02   3.19379985e-01
  -5.47959983e-01   1.97659999e-01   8.97770002e-02   1.26279995e-01
  -3.52310017e-02  -2.45920002e-01  -1.30390003e-01   3.15950006e-01
   7.25120008e-02  -1.19350001e-03   5.32959998e-01   1.97050005e-01
  -2.93260008e-01  -3.10740005e-02   1.86829999e-01   2.10660007e-02
  -1.45049999e-02   5.93950003e-02  -3.48839998e-01  -2.32510000e-01
   2.43689999e-01   5.76899992e-03  -2.04009995e-01  -1.07060000e-01
  -5.06529994e-02  -7.20589980e-02   2.33300000e-01  -9.27010030e-02
  -2.31240004e-01  -3.67370009e-01   3.13459992e-01   1.79600000e-01
  -1.76960006e-01   3.25639993e-01

In [7]:
books_train = pd.read_csv('C:/Users/Анна/Dropbox/ML_sent/books_train.tsv', sep='\t', index_col=0)
films_train = pd.read_csv('C:/Users/Анна/Dropbox/ML_sent/films_train.tsv', sep='\t', index_col=0)

In [8]:
books_train.head()

,lemmatized,X_train,y_train
0,отличный пример боев фантастика нет такой пере...,отличный пример боевой фантастики нет таких пе...,9
1,читать данный произведение в журнальный вариан...,читала данное произведение в журнальном вариан...,10
2,если бы я прочитать этот книга в начало го век...,если бы я прочитал эту книгу в начале го века ...,6
3,я книга очень понравиться заставить глубокий п...,мне книга очень понравилась заставила глубже п...,10
4,очень хотеть полюбить,очень хочу полюбить,10


In [9]:
films_train.head()

,lemmatized,X_train,y_train
0,и опять два часть худой один если продолжить в...,и опять вторая часть хуже первой если продолжа...,7
1,романтический мелодрама по книга питер майль п...,романтическая мелодрама по книге питера майля ...,7
2,ну вроде весь ничто и актёр хороший и сюжет чт...,ну вроде всё ничего и актеры хорошие и сюжет ч...,5
3,посмотреть один раз можно,посмотреть один раз можно,7
4,пока до премьера очередной часть сумерки ещё д...,пока до премьеры очередной части сумерек еще д...,6


In [10]:
from keras.preprocessing.text import Tokenizer

Using TensorFlow backend.


In [11]:
tokenizer_books = Tokenizer()
tokenizer_books.fit_on_texts(books_train.lemmatized)

In [12]:
word_index = tokenizer_books.word_index
len(word_index)

18222

In [13]:
vocabular = list(word_index.keys())

In [20]:
#какая строчка из матрицы соответствует каждому слову из словаря
embedding_matrix = np.zeros((len(word_index), 300)) 
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [15]:
import embedding

In [21]:
embeddings = embedding.Embedding(embedding_matrix, vocabular, normalize=True)

In [22]:
embedding_matrix.shape

(18222, 300)

In [23]:
from socialsent import polarity_induction_methods

## Книги. Контекст

__Книга__
(+)
* понравиться
* интересный
* лучший
* великолепный
* обожать
* приятный
* захватывать
* замечательный
* восторженный
* отличный
* любимый
* потрясающий
* супер
* обалденный
* восторг

(-)
* разочарование
* плохой
* банальность ?
* похуже (?)
* скомканному (?)
* ужасный / ужасно
* пресытиться
* худой
* нудно
* извращает
* средний (?)
* облом
* мусорный
* средненький (?)
* перековеркал

In [30]:
POSITIVE_BOOKS = ['понравиться','интересный', 'великолепный', 'обожать', 'захватывать', 'замечательный', 'отличный', 'супер', 'обалденный', 'восторг']
NEGATIVE_BOOKS = ['разочарование', 'плохой', 'банальность', 'ужасный', 'худой', 'нудный', 'извращать', 'средний', 'облом', 'мусорный']

In [52]:
print(embeddings_index['перековеркать'])

KeyError: 'перековеркать'

In [31]:
polarties = polarity_induction_methods.dist(embeddings, POSITIVE_BOOKS, NEGATIVE_BOOKS)

In [33]:
polarties

{'калейдоскоп': -1.0,
 'ущербный': -1.0,
 'овчинников': -1.0,
 'книгоман': -1.0,
 'дьявольски': -1.0,
 'амели': -1.0,
 'кошачий': -1.0,
 'толк': -1.0,
 'визуализировать': -1.0,
 'териия': -1.0,
 'манящий': -1.0,
 'ульянов': -1.0,
 'скора': -1.0,
 'медлительнность': -1.0,
 'увековечить': -1.0,
 'разбирать': -1.0,
 'славный': -1.0,
 'символический': -1.0,
 'экшный': -1.0,
 'взирать': -1.0,
 'вводный': -1.0,
 'добавление': -1.0,
 'въезжать': -1.0,
 'отъезд': -1.0,
 'мрачно': -1.0,
 'местись': -1.0,
 'вникать': -1.0,
 'пацан': -1.0,
 'вплетаться': -1.0,
 'чёткость': -1.0,
 'колесо': -1.0,
 'снег': -1.0,
 'исчезнуть': -1.0,
 'слабеть': -1.0,
 'подражать': -1.0,
 'вблизи': -1.0,
 'шаров': -1.0,
 'задумываешся': -1.0,
 'спецэффект': -1.0,
 'бабуля': -1.0,
 'раб': -1.0,
 'препятствие': -1.0,
 'гарвуд': -1.0,
 'достигнуть': -1.0,
 'стройность': -1.0,
 'паранойя': -1.0,
 'увлекаться': -1.0,
 'проспать': -1.0,
 'пищевой': -1.0,
 'приготовление': -1.0,
 'объяснять': -1.0,
 'тоннель': -1.0,
 'полуп